# Test deployed web application

This notebook pulls some images and tests them against the deployed web application on AKS.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from testing_utilities import to_img, plot_predictions, get_auth, read_image_from
from azureml.core.workspace import Workspace
from azureml.core.webservice import AksWebservice
from dotenv import set_key, get_key, find_dotenv

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

Get the external url for the web application running on AKS cluster.

In [ ]:
ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

Let's retrieve web service.

In [ ]:
aks_service_name = get_key(env_path, 'aks_service_name')
aks_service = AksWebservice(ws, name=aks_service_name)

In [ ]:
aks_service.state

In [ ]:
scoring_url = aks_service.scoring_uri
api_key = aks_service.get_keys()[0]

Pull an image of a Lynx to test it with.

In [ ]:
IMAGEURL = "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/220px-Lynx_lynx_poing.jpg"

In [ ]:
plt.imshow(to_img(IMAGEURL))

In [ ]:

headers = {'Authorization':('Bearer '+ api_key)}
img_data = read_image_from(IMAGEURL).read()
r = requests.post(scoring_url, files={'image':img_data}, headers=headers) # Run the request twice since the first time takes a 
                                                              # little longer due to the loading of the model
%time r = requests.post(scoring_url, files={'image':img_data}, headers=headers)
r.json()

From the results above we can see that the model correctly classifies this as an Lynx. 

Let's try a few more images.

In [ ]:
images = (
    "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/220px-Lynx_lynx_poing.jpg",
    "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/Roadster_2.5_windmills_trimmed.jpg",
    "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/Harmony_of_the_Seas_(ship,_2016)_001.jpg",
)

In [ ]:
results = [requests.post(scoring_url, files={'image': read_image_from(img).read()}, headers=headers) for img in images]

In [ ]:
plot_predictions(images, results)

The labels predicted by our model seem to be consistent with the images supplied.

Next let's quickly check what the request response performance is for the deployed model on AKS cluster.

In [ ]:
image_data = list(map(lambda img: read_image_from(img).read(), images)) # Retrieve the images and data

In [ ]:
timer_results = list()
for img in image_data:
    res=%timeit -r 1 -o -q requests.post(scoring_url, files={'image': img}, headers=headers)
    timer_results.append(res.best)

In [ ]:
timer_results

In [ ]:
print('Average time taken: {0:4.2f} ms'.format(10**3 * np.mean(timer_results)))

We have tested that the model works and we can now move on to the [next notebook to get a sense of its throughput](06_SpeedTestWebApp.ipynb).